In [83]:
from io import BytesIO,StringIO
from zipfile import ZipFile
from urllib.request import urlopen
from urllib.error import HTTPError
import datetime
import redis
import pandas as pd
r = redis.from_url(os.environ.get("REDIS_URL",'redis:'))
d = datetime.date.today() 
csvData = ""
if d.isoweekday() in range(1, 6):
    file_url = "https://www.bseindia.com/download/BhavCopy/Equity/EQ"+d.strftime("%d%m%y")+"_CSV.ZIP"
#     file_url = "https://www.bseindia.com/download/BhavCopy/Equity/EQ090819_CSV.ZIP"
    try:
        resp = urlopen(file_url)
    except HTTPError as e:
        print(e)
    else:
        zipfile = ZipFile(BytesIO(resp.read()))
        for line in zipfile.open(zipfile.namelist()[0]).readlines():
            csvData+=line.decode('utf-8')

In [84]:
data = pd.read_csv(StringIO(csvData), sep=",")

In [85]:
data.head(10)

,SC_CODE,SC_NAME,SC_GROUP,SC_TYPE,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,NO_TRADES,NO_OF_SHRS,NET_TURNOV,TDCLOINDI
0,500002,ABB LTD.,A,Q,1411.50,1422.60,1388.80,1397.70,1388.80,1396.30,244,1160,1630199.0,NaN
1,500003,AEGIS LOGIS,A,Q,196.65,202.10,195.00,199.00,199.60,192.65,473,9841,1945870.0,NaN
2,500008,AMAR RAJA BA,A,Q,649.05,655.35,621.60,627.60,627.60,645.05,1287,56341,36044463.0,NaN
3,500009,A.SARABHAI,X,Q,11.55,11.93,11.51,11.83,11.90,11.53,46,12871,150785.0,NaN
4,500010,HDFC,A,Q,2199.00,2239.85,2180.00,2211.90,2211.90,2181.20,4853,122789,273140591.0,NaN
5,500012,ANDHRA PETRO,X,Q,42.85,43.50,41.80,43.10,43.10,42.20,152,22316,956006.0,NaN
6,500013,ANSAL INFRAS,B,Q,3.99,4.00,3.90,3.93,3.95,3.81,59,29467,116764.0,NaN
7,500014,APPLE FINANC,XT,Q,1.39,1.45,1.39,1.45,1.45,1.39,32,8271,11985.0,NaN
8,500020,BOM DYEING,A,Q,73.00,74.60,71.70,71.95,71.95,71.25,2613,210642,15360204.0,NaN
9,500023,ASIANHOTNR,B,Q,93.00,93.00,92.55,92.55,92.55,90.60,38,136,12617.0,NaN


In [86]:
data.sort_values("OPEN", axis=0, ascending=False, inplace=True, kind='quicksort', na_position='last')

In [87]:
dataDict = data[['SC_NAME', 'OPEN', 'HIGH', 'LOW', 'CLOSE']].to_dict('records')

In [88]:
dataDict[0]

{'SC_NAME': '85FFSPL27   ',
 'OPEN': 9500000.0,
 'HIGH': 9500000.0,
 'LOW': 9500000.0,
 'CLOSE': 9500000.0}

In [89]:
SC_CODEList = data[['SC_CODE']].values.tolist()
SC_NameList = data[['SC_NAME']].values.tolist()

In [90]:
top10Keys = SC_CODEList[:10]

In [91]:
top10Keys.reverse()

In [92]:
r.lpush('top10Keys', *[item[0] for item in top10Keys])

20

In [93]:
dictionary = {index[0]: value for index,value in zip(SC_CODEList,dataDict)}

In [94]:
# dictionary

In [95]:
with r.pipeline() as pipe:
    for SC_CODE, item in dictionary.items():
        pipe.hmset(SC_CODE, item)
        pipe.execute()

In [96]:
r.bgsave()

True

In [97]:
for SC_CODE, item in dictionary.items():
    r.hmset(SC_CODE, item)

In [98]:
# r.keys()

In [99]:
def makeDictionary(names,codes):
    returnDict = {}
    for name,code in zip(names,codes):
        returnDict[name[0]] = code[0]
    return returnDict

In [100]:
r.mset(makeDictionary(SC_NameList,SC_CODEList))

True

In [78]:
r.mset({"date":d.strftime("%d-%m-%Y")})

True

In [50]:
r.mget("date")[0].decode()

'10-08-2019'

In [ ]:
break

# ---------------------------------------------------------------------------------------------------------------

# Case1

In [ ]:
TTFParse = r.lrange('top10Keys', 0, 10)

In [ ]:
TTFParse.reverse()

In [ ]:
answerList = []
for key in TTFParse:
    answerList.append(r.hmget(key.decode(), "SC_NAME", "OPEN", "HIGH", "LOW", "CLOSE"))

In [ ]:
for row in answerList:
    print(row[0].decode()+" "+row[1].decode()+" "+row[2].decode()+" "+row[3].decode()+" "+row[4].decode())

In [ ]:
break

# Case2

In [ ]:
scname = "HDFC"

In [ ]:
matchingNames = r.scan_iter("*"+scname.upper()+"*")

In [ ]:
answerList = []
for name in matchingNames:
    answerList.append(r.hmget(r.mget(name.decode())[0].decode(), "SC_NAME", "OPEN", "HIGH", "LOW", "CLOSE"))

In [ ]:
for row in answerList:
    print(row[0].decode()+" "+row[1].decode()+" "+row[2].decode()+" "+row[3].decode()+" "+row[4].decode())

In [ ]:
break

# Flush Redis

In [38]:
r.flushall()

True